**package install:**

In [1]:
!pip install moviepy assemblyai elevenlabs pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00


**import modules/libraries:**

In [2]:
import os
import io
import assemblyai as aai
from moviepy.editor import VideoFileClip, AudioFileClip
from elevenlabs import ElevenLabs
from pydub import AudioSegment

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':

  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)

  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)

  elif re.match('(flt)p?( \(default\))?$', token):

  elif re.m

**API KEYS:**

In [3]:
ASSEMBLYAI_API_KEY = "your_api_key"
ELEVENLABS_API_KEY = "your_api_key"

**INPUT/OUTPUT VIDEO PATH ASSIGN:**

In [4]:
original_video_path = "/content/drive/MyDrive/video for dubbing podcast/new input video.mp4"
output_audio_path   = "/content/drive/MyDrive/video for dubbing podcast/final_dubbed_audio3.wav"
output_video_path   = "/content/drive/MyDrive/video for dubbing podcast/final_dubbed_video3.mp4"

**SELECT VOICE ID:**

In [5]:
speaker_voices = {
    "A": "pNInz6obpgDQGcFmaJgB",
    "B": "TxGEqnHWrfWFTfGW9XjX",
    "SPEAKER_0": "pNInz6obpgDQGcFmaJgB",
    "SPEAKER_1": "TxGEqnHWrfWFTfGW9XjX"
}

**Extract audio from video**

In [6]:
video = VideoFileClip(original_video_path)
audio_path = "extracted_audio.wav"
video.audio.write_audiofile(audio_path, verbose=False, logger=None)
print(f" Video duration: {video.duration:.2f}s")

 Video duration: 62.23s


**Transcribe + diarize using AssemblyAI:**

In [7]:
aai.settings.api_key = ASSEMBLYAI_API_KEY
transcriber = aai.Transcriber()
config = aai.TranscriptionConfig(speaker_labels=True, speakers_expected=2)
transcript = transcriber.transcribe(audio_path, config)

segments = transcript.utterances
print(f"Total segment detect: {len(segments)}")

Total segment detect: 40


**Prepare ElevenLabs client:**

In [8]:
client = ElevenLabs(api_key=ELEVENLABS_API_KEY)

**summary diarized segments:**

**Generate dubbed audio per segment:**

In [ ]:
!pip install moviepy elevenlabs pydub

In [ ]:
import time

In [9]:
total_duration_ms = int(video.duration * 1000)
final_dub_audio = AudioSegment.silent(duration=total_duration_ms)

for seg in segments:
    speaker = seg.speaker
    text = seg.text.strip()
    start_ms = int(seg.start)
    end_ms   = int(seg.end)
    duration_ms = max(200, end_ms - start_ms)
    voice_id = speaker_voices.get(speaker, "pNInz6obpgDQGcFmaJgB")

    print(f"\n Generating {speaker} ({start_ms}-{end_ms} ms | {duration_ms/1000:.2f}s): '{text}'")

    seg_audio = None
    attempt = 0

    while attempt < 2 and seg_audio is None:
        try:
            audio_stream = client.text_to_speech.convert(
                voice_id=voice_id,
                model_id="eleven_multilingual_v2",
                text=text
            )

            audio_bytes = b"".join(audio_stream)
            if not audio_bytes:
                raise ValueError("Empty stream received from ElevenLabs")

            seg_audio = AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3")

        except Exception as e:
            attempt += 1
            print(f" Error (attempt {attempt}) for {speaker}: {e}")
            time.sleep(2)

    if seg_audio is None:
        print(f" ElevenLabs failed for '{text}', inserting {duration_ms}ms silence.")
        seg_audio = AudioSegment.silent(duration=duration_ms)

    final_dub_audio = final_dub_audio.overlay(seg_audio, position=start_ms)


 Generating A (160-880 ms | 0.72s): 'What is your name?'

 Generating B (1120-2160 ms | 1.04s): 'My name is Emma.'

 Generating A (2400-3280 ms | 0.88s): 'How old are you?'

 Generating B (3520-4880 ms | 1.36s): 'I'm 25 years old.'

 Generating A (5200-6160 ms | 0.96s): 'Where are you from?'

 Generating B (6400-7600 ms | 1.20s): 'I am from Germany.'

 Generating A (7760-8720 ms | 0.96s): 'What do you do?'

 Generating B (9040-9920 ms | 0.88s): 'I'm a student.'

 Generating A (10160-11120 ms | 0.96s): 'Are you married?'

 Generating B (11360-12400 ms | 1.04s): 'No, I'm single.'

 Generating A (12640-14160 ms | 1.52s): 'Do you have any siblings?'

 Generating B (14560-16000 ms | 1.44s): 'Yes, I have one brother.'

 Generating A (16400-17760 ms | 1.36s): 'What is your favorite color?'

 Generating B (18000-19520 ms | 1.52s): 'My favorite color is blue.'

 Generating A (19840-21120 ms | 1.28s): 'What time do you wake up?'

 Generating B (21200-22800 ms | 1.60s): 'I wake up at 7 o'. Clock

In [10]:
dubbed_ms = len(final_dub_audio)
video_ms = int(video.duration * 1000)

if dubbed_ms < video_ms:
    final_dub_audio += AudioSegment.silent(duration=video_ms - dubbed_ms)
elif dubbed_ms > video_ms:
    final_dub_audio = final_dub_audio[:video_ms]

In [11]:
final_dub_audio.export(output_audio_path, format="wav")
print(f"\n Dubbed audio saved at: {output_audio_path}")


 Dubbed audio saved at: /content/drive/MyDrive/video for dubbing podcast/final_dubbed_audio3.wav


**Merge dubbed audio back into video**

In [12]:
dub_audio = AudioFileClip(output_audio_path)
final_video = video.set_audio(dub_audio)
final_video.write_videofile(output_video_path, codec="libx264", audio_codec="aac")

print(f"\n Final dubbed video saved at: {output_video_path}")

Moviepy - Building video /content/drive/MyDrive/video for dubbing podcast/final_dubbed_video3.mp4.
MoviePy - Writing audio in final_dubbed_video3TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/video for dubbing podcast/final_dubbed_video3.mp4



t: 100%|█████████▉| 1863/1867 [00:25<00:00, 54.23it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/video for dubbing podcast/new input video.mp4, 691200 bytes wanted but 0 bytes read,at frame 1864/1867, at time 62.13/62.23 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/video for dubbing podcast/final_dubbed_video3.mp4

 Final dubbed video saved at: /content/drive/MyDrive/video for dubbing podcast/final_dubbed_video3.mp4
